In [32]:
import time
import json
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from sklearn.linear_model import LogisticRegression
import MLStrategy as mls
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [33]:
# Import prepared dictionary which contains average spreads for all assets
with open('Data\spreads.json', 'r') as fp:
    spreads = json.load(fp)

# Prepare instruments list from spreads file
instruments_list = []
for key in spreads:
    instruments_list.append(key)
instruments_list = instruments_list[1:]

In [35]:
timeframes = ["M15","M5","M3"]
i = 0
j = len(instruments_list)

In [38]:
for symbol in instum:
    start_time = time.time()
    i += 1
    cost = spreads[symbol]/2
    test_outcome = pd.DataFrame(columns=['Symbol','Przedział','Koszt','Lag','Celność','Liczba transakcji','Wynik(%)','Wynik w stosunku do benchmarku(%)'])
    for timeframe in timeframes:
        ml = mls.LogisticStrategy(symbol, timeframe, cost)
        for lags in range(3, 11):
            performance, outperformance = ml.test_strategy(lags)
            performance = round(performance*100-100,2)
            outperformance = round(outperformance*100,2)
            #print("Lags: {}, perf: {}, out/under: {}".format(lags, performance, outperformance))
            hits = np.sign(ml.results.returns * ml.results.pred).value_counts()
            hit_ratio = round(hits[1.0] / sum(hits) * 100,2)
            trades = int(ml.results.trades.sum())
            #print('Hit Ratio = {} with {} trades'.format(hit_ratio,trades))
            outcome = pd.DataFrame([[symbol,timeframe,cost,lags,hit_ratio,trades,performance,outperformance]],
                                columns=['Symbol','Przedział','Koszt','Lag','Celność','Liczba transakcji','Wynik(%)','Wynik w stosunku do benchmarku(%)'])
            test_outcome = test_outcome.append(outcome)
    test_outcome.to_csv('Wyniki\ML\perfs_{}.csv'.format(symbol),index=True)
    print("Part {}/{} is done. Calculating {} took {}min".format(i,j, symbol,round(time.time() - start_time)/60,2))
print("COMPLETED")

Part 1/122 is done. Calculating EUR_PLN took 0.75min
COMPLETED


### Due to errors occured in calculation it was decided to drop the symbols which caused them

In [88]:
drop_list = ['XAU_USD','XAU_AUD','XAU_CAD','XAU_CHF','XAU_GBP','XAU_JPY','XAU_NZD','XPD_USD','XAG_AUD']

In [89]:
# Use list comprehensions
new_list = [symbol for symbol in instruments_list if symbol not in drop_list]

In [90]:
len(new_list)

113

### Reading saved information to find best performing asset

In [91]:
all_data = pd.DataFrame(columns=['Symbol','Przedział','Koszt','Lag','Celność','Liczba transakcji','Wynik(%)','Wynik w stosunku do benchmarku(%)'])

In [92]:
for symbol in new_list:
    dane = pd.read_csv('Wyniki\ML\perfs_{}.csv'.format(symbol),index_col=['Unnamed: 0'])
    all_data = all_data.append(dane)
all_data.to_csv('Wyniki\ML\_allperformances.csv',index=True)

In [93]:
all_data

,Symbol,Przedział,Koszt,Lag,Celność,Liczba transakcji,Wynik(%),Wynik w stosunku do benchmarku(%)
0,AUD_CAD,M15,0.000115,3,50.96,1886,-13.39,-13.90
0,AUD_CAD,M15,0.000115,4,50.55,2006,-12.98,-13.30
0,AUD_CAD,M15,0.000115,5,50.82,2142,-14.89,-15.48
0,AUD_CAD,M15,0.000115,6,50.74,2222,-16.22,-16.67
0,AUD_CAD,M15,0.000115,7,51.18,2334,-15.89,-16.40
...,...,...,...,...,...,...,...,...
0,ZAR_JPY,M3,0.005940,6,41.41,16438,-100.00,-100.55
0,ZAR_JPY,M3,0.005940,7,41.74,16870,-100.00,-100.53
0,ZAR_JPY,M3,0.005940,8,41.42,17474,-100.00,-100.44
0,ZAR_JPY,M3,0.005940,9,41.60,4768,-100.00,-100.61


In [94]:
losses = all_data.where(all_data['Wynik(%)']==-100).dropna()

In [95]:
gainers = all_data.where(all_data['Wynik(%)']>0).dropna()

In [96]:
gainers.where(gainers['Liczba transakcji']!=0).dropna()

,Symbol,Przedział,Koszt,Lag,Celność,Liczba transakcji,Wynik(%),Wynik w stosunku do benchmarku(%)
0,AUD_CHF,M5,0.000135,3,49.77,48,2.55,-0.87
0,AUD_CHF,M5,0.000135,4,49.83,100,2.11,-1.45
0,AUD_CHF,M5,0.000135,5,49.84,108,2.30,-1.14
0,AUD_CHF,M5,0.000135,6,49.84,108,2.17,-1.09
0,AUD_CHF,M5,0.000135,7,49.83,140,1.97,-1.28
0,AUD_CHF,M5,0.000135,8,49.86,308,0.08,-3.50
0,AUD_CHF,M3,0.000135,3,48.91,92,2.27,-1.19
0,AUD_SGD,M15,0.000190,3,50.44,56,0.28,-1.18
0,AUD_USD,M15,0.000070,3,50.29,268,0.38,-0.46
0,AUD_USD,M15,0.000070,5,50.61,472,1.01,0.10


In [23]:
# Creating table to store only the highest Win(%) models
total_outcome = pd.DataFrame(columns=['Symbol','Przedział','Koszt','Lag','Celność','Liczba transakcji','Wynik(%)','Wynik w stosunku do benchmarku(%)'])

In [24]:
# For each symbol return best value and next filter only the one with positive number
for symbol in new_list:
    dane = pd.read_csv('Wyniki\ML\perfs_{}.csv'.format(symbol),index_col=['Unnamed: 0'])
    # Not interested in lines with no transactions
    df = dane.where(dane['Liczba transakcji']!=0).dropna() 
    # Only pick best outcome for every symbol
    best_line = df.where(df['Wynik(%)']==df['Wynik(%)'].max()).dropna().head(1) 
    total_outcome = total_outcome.append(best_line)
total_outcome.to_csv('Wyniki\ML\performances.csv',index=True)

In [28]:
total_outcome

,Symbol,Przedział,Koszt,Lag,Celność,Liczba transakcji,Wynik(%),Wynik w stosunku do benchmarku(%)
0,AUD_CAD,M15,0.000115,4.0,50.55,2006.0,-12.98,-13.30
0,AUD_CHF,M5,0.000135,3.0,49.77,48.0,2.55,-0.87
0,AUD_HKD,M15,0.000715,3.0,50.88,12.0,-0.02,-0.91
0,AUD_JPY,M5,0.008715,3.0,49.99,220.0,-84.87,-88.70
0,AUD_NZD,M15,0.000145,3.0,49.61,2358.0,-28.93,-28.50
...,...,...,...,...,...,...,...,...
0,USD_TRY,M15,0.007340,10.0,49.97,4.0,-2.28,-2.79
0,USD_ZAR,M5,0.006105,3.0,50.10,1104.0,-99.88,-102.61
0,WTICO_USD,M3,0.015020,3.0,47.19,252.0,-97.20,-126.73
0,WHEAT_USD,M3,0.005325,6.0,42.36,52.0,-26.64,-28.89


In [30]:
# Filter only positive returns 
total_outcome.where(total_outcome['Wynik(%)']>=0).dropna() 

,Symbol,Przedział,Koszt,Lag,Celność,Liczba transakcji,Wynik(%),Wynik w stosunku do benchmarku(%)
0,AUD_CHF,M5,0.000135,3.0,49.77,48.0,2.55,-0.87
0,AUD_SGD,M15,0.000190,3.0,50.44,56.0,0.28,-1.18
0,AUD_USD,M15,0.000070,5.0,50.61,472.0,1.01,0.10
0,XCU_USD,M3,0.000875,6.0,45.35,8.0,16.11,-1.43
0,GBP_USD,M5,0.000085,4.0,48.84,92.0,1.17,-1.19
0,SUGAR_USD,M3,0.000135,3.0,47.31,770.0,3.35,-8.02
